# About this Notebook

Data Wrangling can be defined as the process to:
* gather
* assess
* clean
data in support of data analytics tasks.

This notebook contains the code to extract data related to Master Swim Performances and save it in a local format that supports Data Exploration activities.

## Supported Data Sources

Currently data is only extracted from the Canadian swimming site that collects the results for Masters Swimmers.

## Error Handling

When I created the meet list for a complete season, I found that some meets don't have results posted

Right now, my code assumes that results will exist.  This will need to be updated

In [1]:
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
from random import randint
import logging
import utilities as util

In [2]:
#set up the logger and initially set the logging level to debug
# since I am using a notebook, I am not going to capture the log in a file
logging.basicConfig(level=logging.DEBUG)

# Canadian Master Swimming Results Collection 

Starting with the information I know how to access from personal experience.

## Gather - www.swimming.ca
The following code will scrape the available data from the www.swimming.ca website for Masters Swimmers. The code is based on the website structure that exists during the time period Aug - Oct 2020.

The website has a layered approach to finding results.
1. Navigate to the page that has a list of meet-results
2. Filter the list of meet results by **Season**, **Province**, **Month**
3. Select the meet of interest in the filtered list
4. At this point it is possible to see all the results for each club that participated, or the results by event.  I will be collecting all the results by participating club.

I did contact Swim Canada to see if there was an API that would allow me to use an approach other than web scraping to get the results, but so far they have not been able to support my request.

For the parameters **Seasion**, **Province** and **Month**, there are few points to note:
* Season - the swim season in Canada is split over 2 calendar years and runs Sep to Aug.  So the swim season 2019/2020 will be the period 1 Sep 2019 to 31 Aug 2020.  The value passed to the **Season** parameter is the last 2 numbers in the second part of the season identifier.  So to access swim meets from Nov 2019, the season parameter will be 20 ('season'='20')
* Month - the month must be specified as a number from 1 to 12.  It is not possible to have all the months in a single season being available at one time
* Province - It is possible to filter by province/territory or use all provinces. To access an individual province/territory, the provinces/territories are numbered from 1 to 11 in alphabetical order.  So to get only Alberta, 'province'='1' and to get Yukon, 'province'='11'.  The supported Territories are Northwest Territories and Yukon.  For all provinces, no number is provided to province (province = '')

Since I am only interested in Masters results, when looking at the meet list, I will look for MEET TYPE  of Masters.  This will miss some swimmers that compete in non-Masters meets as an Open Master.  I plan to come back and find these results after collecting the Masters Meets

The only URL specified in my code is the URL associated with step 1 above, navigating to the meet-results page.  The subsequent URLs will be recovered from the information available when the various filters are applied.

In [ ]:
# URLs of interest
url_meetlist ='https://www.swimming.ca/en/events-results/meet-results/'

In [ ]:
#change the logging level
logging.getLogger().setLevel(logging.INFO)

## Get Meet Info
The next section of code is used to retrieve meet information and write the results to a csv.  One csv per season

### Seasons Processed
To see what seasons have been processed, look at the csv's in the same file folder as this notebook.  Best effort list as I process a season:
* 2019-2020
* 2018-2019
* 2017-2018
* 2016-2017

For each season:

* update the season value and csv name in the dictionary
* change the returned list to a datafram
* write the dataframe to a csv

In [3]:
season_dict = {}
season_dict['season'] = '18'
season_dict['meet_file'] = 'meets_2017_2018.csv'
season_dict['club_file'] = 'clubs_2017_2018.csv'
season_dict['swimmers_file'] = 'swimmers_2017_2018.csv'
season_dict['race_file'] = 'races_2017_2018.csv'

In [ ]:
# function to get meet list
def getMeetList(season, month, base_url):
    #create param list for the URL
    call_params = {'season': season, 'province':'', 'month': month}
    
    #get the page after a random delay
    #I think the crawl delay is 10 seconds, so while it will take longer,
    #I will set a random delay of 10-14 seconds
    sleepTime = randint(10,14)
    logging.debug("sleep time is %i seconds", sleepTime)
    time.sleep(sleepTime)
    response=requests.get(base_url, params=call_params)
    logging.debug('%s', response.url)
    
    #parse the page and create the list
    #use Beautiful Soup to parse the returned page
    meetList_resp = BeautifulSoup(response.text, 'lxml')
    
    rtnList = []
    for item in meetList_resp.find_all('tr'):
        if item.contents[5].contents[0] == "Masters":
            ## need to handle the case where there was a meet, but there are no results
            
            temp_dict = {}
            try:
                temp_dict['meet_date'] = item.contents[0].contents[0].contents[0]
            except:
                temp_dict['meet_date'] = None
                
            try:    
                temp_dict['meet_url'] = item.contents[1].a.attrs['href']
            except:
                temp_dict['meet_url'] = None
                
            try:    
                temp_dict['meet_prov'] = item.contents[2].contents[0]
            except:
                temp_dict['meet_prov'] = None
                
            try:
                temp_dict['meet_host'] = item.contents[3].contents[0].contents[0]
            except:
                temp_dict['meet_host'] = None
                
            try:
                temp_dict['meet_course'] = item.contents[4].contents[0]
            except:
                temp_dict['meet_course'] = None
                
            try:
                temp_dict['meet_type'] = item.contents[5].contents[0]
            except:
                temp_dict['meet_type'] = None
                
            try:
                temp_dict['meet_status'] = item.contents[6].contents[0]
            except:
                temp_dict['meet_status'] = None
                
            rtnList.append(temp_dict)
    
    logging.debug("Number of meets: %i", len(rtnList))
    #return the list of meets
    return rtnList

In [ ]:
MeetListFn = []

#get the meet list for a season
monthList = ['9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']
for mon in monthList:
    logging.info("month being processed %s", mon)
    tempList = getMeetList(season_dict['season'], mon, url_meetlist)
    for item in tempList:
        MeetListFn.append(item)

In [ ]:
len(MeetListFn)

In [ ]:
#create a dataframe and write the dataframe to a csv
df = pd.DataFrame(MeetListFn)
df.head()

In [ ]:
df.to_csv(season_dict['meet_file'], index=False)

## Get teams participating in meet info

Next section will get the team lists for each meet and write the results to a csv

In [ ]:
# read in the meet list file
#df_meet = pd.read_csv('meets_2018_2019.csv')
df_meet = pd.read_csv(season_dict['meet_file'])
df_meet.head()

In [ ]:
df_meet.shape

In [ ]:
#make a list of the meet urls
meetURLS = df_meet['meet_url'].tolist()

In [ ]:
#change the logging level
#logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
def getTeamList(meet_url):
    
    logging.debug(meet_url)
    sleepTime = randint(10,14)
    logging.debug("sleep time is %i seconds", sleepTime)
    time.sleep(sleepTime)
    response=requests.get(meet_url)
    logging.debug('%s', response.url)
        
    #parse the swim club list page
    #use Beautiful Soup to parse the returned page
    clubList_resp = BeautifulSoup(response.text, 'lxml')

    rtnList = []
    for item in clubList_resp.find_all('option'):
        temp_dict = {}
        if "Events" in item.contents[0]:
            logging.debug("Did I get here?")
            break
        elif "Participants" not in item.contents[0]:
            try:
                temp_dict['club_res_url'] = item.attrs['data-href']
            except:
                temp_dict['club_res_url'] = None
            try:
                temp_dict['club_number'] = item.attrs['value']
            except:
                temp_dict['club_number'] = None
            try:
                temp_dict['club_name'] = item.contents[0]
            except:
                temp_dict['club_name'] = None
            rtnList.append(temp_dict)

    return rtnList

In [ ]:
clubListFn = []

for meet in meetURLS:
    logging.info("meet URL %s", meet)
    tempList = getTeamList(meet)
    for item in tempList:
        clubListFn.append(item)


In [ ]:
#create a dataframe and write the dataframe to a csv
club_df = pd.DataFrame(clubListFn)
club_df.head()

In [ ]:
#club_df.to_csv('clubs_2018_2019.csv', index=False)
club_df.to_csv(season_dict['club_file'], index=False)

## Get swimmer and race info

The next section will get the results for meet and save the information to two files.  One will contain the information on the swimmer and one will contain the informaiton on the races

In [4]:
season_dict

{'season': '18',
 'meet_file': 'meets_2017_2018.csv',
 'club_file': 'clubs_2017_2018.csv',
 'swimmers_file': 'swimmers_2017_2018.csv',
 'race_file': 'races_2017_2018.csv'}

In [5]:
#get the csv containing the club links
swims_df = pd.read_csv(season_dict['club_file'])

In [6]:
swims_df.shape

(539, 3)

In [7]:
swims_df.head()

,club_name,club_number,club_res_url
0,La Vague de Brossard,73992.0,https://www.swimming.ca/en/meet/604494/?factio...
1,McGill Masters,76063.0,https://www.swimming.ca/en/meet/604494/?factio...
2,E.N. Longueuil,72351.0,https://www.swimming.ca/en/meet/604494/?factio...
3,Maitres A Contre-Courant,74019.0,https://www.swimming.ca/en/meet/604494/?factio...
4,Club Aquatique Montreal,72346.0,https://www.swimming.ca/en/meet/604494/?factio...


In [ ]:
len(swims_df['club_name'].unique())

In [8]:
#make a list of urls for the results from each club
clubURLS = swims_df['club_res_url'].tolist()

In [9]:
clubURLS[0]

'https://www.swimming.ca/en/meet/604494/?faction=73992'

In [10]:
#test my utils file
tmp_sw, tmp_rc = util.getRaceResults(clubURLS[0])

DEBUG:root:https://www.swimming.ca/en/meet/604494/?faction=73992
DEBUG:root:sleep time is 10 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/604494/?faction=73992 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/604494/?faction=73992
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:first row of table, ignore


In [12]:
tmp_sw[0]

{'sw_url': 'https://www.swimming.ca/en/swimmer/4209372/',
 'sw_id': '4209372',
 'sw_name': 'Aviles, Carlos',
 'sw_yob': '1975',
 'sw_gender': 'male'}

In [13]:
tmp_rc[0]

{'rc_dist_stroke': '50m Freestyle',
 'rc_round': 'Final',
 'rc_time': '29.04',
 'rc_course': '25m',
 'sw_id': '4209372',
 'sw_yob': '1975',
 'sw_gender': 'male'}

In [ ]:
len(clubURLS)

In [ ]:
r3_url = clubList[0]['club_res_url']
r3 = requests.get(r3_url)
temp_r3 = BeautifulSoup(r3.text, 'lxml')

In [ ]:
temp_table = temp_r3.find_all("table")
len(temp_table)

In [ ]:
#todo: use this method with the previously define method rather than having the code repeated multiple times
def scrapePage(url):
    #get the page with the results for the club
    logging.debug(url)
    sleepTime = randint(10,14)
    logging.debug("sleep time is %i seconds", sleepTime)
    time.sleep(sleepTime)
    response=requests.get(url)
    logging.debug('%s', response.url)
    
    #use Beautiful Soup to parse the returned page
    resp = BeautifulSoup(response.text, 'lxml')
    
    #return the page
    return resp

In [ ]:
def parseSwimmerInfo(tableRow):
    
    #create return dictionary
    sw_dict = {}
    
    #parse the row
    try:
        sw_dict['sw_url'] = tableRow.find('a').attrs['href']
    except:
        sw_dict['sw_url'] = None 
    try:
        #sw_id = tableRow.find('a').attrs['href'].split('/')[5]
        sw_dict['sw_id'] = tableRow.find('a').attrs['href'].split('/')[5]
    except:
        sw_dict['sw_id'] = None
    try:
        sw_dict['sw_name'] = tableRow.find('a').contents[0]
    except:     
        sw_dict['sw_name'] = None
    try:
        sw_dict['sw_yob'] = tableRow.find('th').contents[1][3:7]
    except:    
        sw_dict['sw_yob'] = None
        
    #return the dictionary
    return sw_dict
        

In [ ]:
#for testing, I am going to get 2 pages - one with relays and one without to make sure I can handle either
url_relays = 'https://www.swimming.ca/en/meet/598224/?faction=73928'
url_no_relays = 'https://www.swimming.ca/en/meet/597428/?faction=74044'
relay_pg = scrapePage(url_relays)
no_relay_pg = scrapePage(url_no_relays)


In [ ]:
def parseRaceInfo(tableRow):
    
    #create return dictionary
    rc_dict = {}
    
    #rc_dict['sw_id'] = sw_id
    #rc_dict['sw_yob'] = sw_yob
    #try:
    #    rc_dict['sw_gender'] = tableRow.contents[0].find('a').attrs['data-query-gender']
    #except:
    #    rc_dict['sw_gender'] = None
    try:
        rc_dict['rc_dist_stroke'] = tableRow.contents[0].find('span').find('a').contents[0]
    except:
        rc_dict['rc_dist_stroke'] = None
    try:
        rc_dict['rc_round'] = tableRow.contents[1].contents[0]
    except:
        rc_dict['rc_round'] = None
    try:
        rc_dict['rc_time'] = tableRow.contents[3].contents[0]
    except:
        rc_dict['rc_time'] = None
    try:
        rc_dict['rc_course'] = tableRow.contents[4].find('abbr').contents[0]
    except:
        rc_dict['rc_course'] = None
    
    #print(rc_dict)
    return rc_dict

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
##### Remember to change this back to use the URL as input
def getRaceResults(club_url):
#def getRaceResults(raceList_resp):
    
    ##### Remember to uncomment this to scrape the page
    #get the page with the results for the club
    raceList_resp = scrapePage(club_url)

    #create a list of tables on the page
    temp_table = raceList_resp.find_all('table')
    logging.debug("number of tables %i", len(temp_table))
    
    #create a gender table to use based on which table is being parsed
    gender_dict = {0:"male", 1:"female", 2:"relay"}
    
    #create lists to hold swimmer and race info
    swimmer_list = []
    race_list = []
    
    #if 3 or more tables, there are results on the page
    if len(temp_table) < 3:
        logging.info("no results on the page")
        #add code to put all nulls in the return value
    else:
        for tt in range(len(temp_table)-2):
            logging.debug("on table %i", tt)
            #set a flag for the first row, want to ignore it
            firstRow = True
            
            #get the gender based on which table is being parsed
            sw_gender = gender_dict[tt]
            
            #create variables to hold the last swimmer id and yob
            sw_id = None
            sw_yob = None
            
            #loop through each row in the table
            for item in temp_table[tt].find_all('tr'):
                temp_sw_dict = {}
                temp_rc_dict = {}
                if firstRow == True:
                    logging.debug("first row of table, ignore")
                    firstRow = False
                elif item.has_attr('class'):
                    #parse the swimmer information
                    temp_sw_dict = parseSwimmerInfo(item)
                    
                    #add the gender
                    try:
                        temp_sw_dict['sw_gender'] = sw_gender
                    except:
                        temp_sw_dict['sw_gender'] = None
                        
                    #make the swimmer id and yob available
                    sw_id = temp_sw_dict['sw_id']
                    sw_yob = temp_sw_dict['sw_yob']
                        
                    #append to the return list
                    swimmer_list.append(temp_sw_dict)
                    
                else:
                    
                    #parse the race information
                    temp_rc_dict = parseRaceInfo(item)
                    
                    #add the swimmer id, yob, gender
                    temp_rc_dict['sw_id'] = sw_id
                    temp_rc_dict['sw_yob'] = sw_yob
                    temp_rc_dict['sw_gender'] = sw_gender
                    
                    #append to the return list
                    race_list.append(temp_rc_dict)
                    
                #else:
                #    logging.debug("expect no information in this row")
                #count += 1
    return swimmer_list, race_list

In [ ]:
test1, test2 = getRaceResults('https://www.swimming.ca/en/meet/618134/?faction=74044')

In [ ]:
sw_list = []
rc_list = []

In [ ]:
for item in clubURLS:
    try:
        if "https" in item:
            tmp_sw_list, tmp_rc_list = getRaceResults(item)
            sw_list += tmp_sw_list
            rc_list += tmp_rc_list
    except:
        print(item, "is not a valid url")

In [ ]:
clubURLS[86]

In [ ]:
len(clubURLS)

In [ ]:
len(clubURLS[87:])

In [ ]:
clubURLS[-1]

In [ ]:
trim_clubURLS = clubURLS[86:]
trim_clubURLS[0]

In [ ]:
trim_clubURLS[1]

In [ ]:
for item in trim_clubURLS[0:4]:
    try:
        if "https" in item:
            print(item, " is valid url")
    except:
        print(item, " is not a valid url")

In [ ]:
for item in trim_clubURLS:
    try:
        if "https" in item:
            tmp_sw_list, tmp_rc_list = getRaceResults(item)
            sw_list += tmp_sw_list
            rc_list += tmp_rc_list
    except:
        print(item, "is not a valid url")

In [ ]:
swim_df = pd.DataFrame(sw_list)
swim_df.head()

In [ ]:
swim_df.shape

In [ ]:
race_df = pd.DataFrame(rc_list)
race_df.head()

In [ ]:
race_df.shape

In [ ]:
swim_df.to_csv(season_dict['swimmers_file'], index=False)
race_df.to_csv(season_dict['race_file'], index=False)

In [ ]:
tmp_sw_list, tmp_rc_list = getRaceResults(relay_pg)

In [ ]:
len(tmp_sw_list)

In [ ]:
tmp_sw_list[0]

In [ ]:
len(tmp_rc_list)

In [ ]:
tmp_rc_list[0]

In [ ]:
sw_list += tmp_sw_list
rc_list += tmp_rc_list

print("sw_list", len(sw_list), "rc_list", len(rc_list))

In [ ]:
tmp_sw_list, tmp_rc_list = getRaceResults(no_relay_pg)

In [ ]:
len(tmp_sw_list)

In [ ]:
len(tmp_rc_list)

In [ ]:
sw_list += tmp_sw_list
rc_list += tmp_rc_list

print("sw_list", len(sw_list), "rc_list", len(rc_list))

## Assess - www.swimming.ca

## Clean - www.swimming.ca

# Development Code

While creating the final functions, I used the following code to explore the webpages to set up the parsing correctly

I don't want to loose it, but it should not be run

### Explore Structure of html pages
In order to recover the information of interest, the way the information is presented on the various html pages needs to be understood.

Once I know how to capture the data of interest, I will create functions to iterate through diffferent seasons, months and meets.

The information I am interested in can be grouped into 3 categories.

**Swimmer Information**
* sw_id_num - unique swimmer id number
* sw_name - swimmers name
* sw_yob - swimmers year of birth

Note that swimmers age is determined by the age on Dec 31st of the year in which the competition happened.

**Meet Information**

* mt_id_num - unique meet id number
* mt_name - meet name
* mt_date - first day of meet
* mt_duration - number of days the swim meet lasted
* mt_pool_name - name of the pool
* mt_pool_address - address of the pool
* mt_sc_lc - indication if meet was a long course or short course meet

**Race Information**
* rc_dist - distance of race - eg 25m, 100m, 400m
* rc_stroke - stroke of race - freestyle, backstroke, butterfly, breaststroke, IM
* rc_time - time it took the swimmer to complete the distance
* rc_round - heat, final, split of longer swim or part of relay
* sw_id_num - unique swimmer id
* mt_id_num - unique meet id

I am also confident that at some point I will want club information, but at this point I'm not sure what I would do with it.

I also have not decided the best way to store the information I scrape.  Initially, I will put it in 3 different csv's, but would like to explore a graph database.  I have not used them before and would like to learn.

In [ ]:
# start with the 2019/2020 season, noting that the season came to an abrupt end in March 2020 because of Covid19
# using November since I know there is a masters meet in Nov 2019
call_params = {'season':'20', 'province':'', 'month':'11'}
#response=requests.get('https://www.swimming.ca/en/events-results/meet-results/', params={'season':'20', 'province':'', 'month':'11'})
response=requests.get('https://www.swimming.ca/en/events-results/meet-results/', params=call_params)
#sanity check that my parameter specification worked as expected
response.url

In [ ]:
#use Beautiful Soup to parse the returned page
meetList_resp = BeautifulSoup(response.text, 'lxml')

In [ ]:
meetList_resp.find_all('tr')

In [ ]:
#find the masters swim meets in the response text
meetList = []
for item in meetList_resp.find_all('tr'):
    if item.contents[5].contents[0] == "Masters":
        temp_dict = {}
        temp_dict['meet_date'] = item.contents[0].contents[0].contents[0]
        temp_dict['meet_url'] = item.contents[1].a.attrs['href']
        temp_dict['meet_prov'] = item.contents[2].contents[0]
        temp_dict['meet_host'] = item.contents[3].contents[0].contents[0]
        temp_dict['meet_course'] = item.contents[4].contents[0]
        temp_dict['meet_type'] = item.contents[5].contents[0]
        meetList.append(temp_dict)
        ##used for debug/development
        #print(item.contents[0].contents[0].contents[0])
        #print(item.contents[1].a.attrs['href'])
        #print(item.contents[2].contents[0])
        #print(item.contents[3].contents[0].contents[0])
        #print(item.contents[4].contents[0])
        #print(item.contents[5].contents[0])
print("number of masters meets", len(meetList))

In [ ]:
meetList[0]

So 8 masters meets were held in Canada in Nov 2019.

Now use the meet_url to get the results for that meet.  The use Beautiful Soup to parse the returned page

In [ ]:
r2_url = meetList[0]['meet_url']
r2 = requests.get(r2_url)
temp_r2 = BeautifulSoup(r2.text, 'lxml')

My next step was to create a list of clubs that participated. I looked through the html on the page and determined the information I was interested in was held in 'option' tags.

In [ ]:
clubList = []
for item in temp_r2.find_all('option'):
    #print(item.contents[0])
    temp_dict = {}
    if "Events" in item.contents[0]:
        #print(item)
        break
    elif "Participants" not in item.contents[0]:
        temp_dict['club_res_url'] = item.attrs['data-href']
        temp_dict['club_number'] = item.attrs['value']
        temp_dict['club_name'] = item.contents[0]
        clubList.append(temp_dict)
        ## used for dev/debug
        #print(item.attrs['data-href'])
        #print(item.attrs['value'])
        #print(item.contents[0])
        
print("number of clubs in the meet", len(clubList))
print("information about each club") 
clubList[0] 

The next step is to get the club results and retrieve the information about each swimmer

In [ ]:
r3_url = clubList[0]['club_res_url']
r3 = requests.get(r3_url)
temp_r3 = BeautifulSoup(r3.text, 'lxml')

After examining the HTML for the per club results page, the results of interest seem to be in the first 4 tables:
* table 0 - men's results
* table 1 - women's results
* table 2 - relay results
* table 3 - meet resource info

In [ ]:
temp_table = temp_r3.find_all("table")
len(temp_table)

In [ ]:
#look at the results for the men's table
count = 0
swimmer_list = []
race_list = []
for item in temp_table[0].find_all('tr'):
    temp_sw_dict = {}
    temp_rc_dict = {}
    if item.has_attr('class'):
        print("new swimmer!!")
        #print("row count", count, "length of info", len(item), "contents", item)
        #print("swimmer url", item.find('a').attrs['href'])
        #print("swimmer id", item.find('a').attrs['href'].split('/')[5])
        #print("swimmer name", item.find('a').contents[0])
        #print("swimmer YOB", item.find('th').contents[1][3:7])
        # need swim id for both the swimmer and the race dictionary
        sw_id = item.find('a').attrs['href'].split('/')[5]
        sw_yob = item.find('th').contents[1][3:7]
        temp_sw_dict['sw_url'] = item.find('a').attrs['href']
        temp_sw_dict['sw_id'] = sw_id
        temp_sw_dict['sw_name'] = item.find('a').contents[0]
        temp_sw_dict['sw_yob'] = sw_yob
        swimmer_list.append(temp_sw_dict)
    #print(count)
    #if item.children[0].has_attr('class'):
    #    print("race time and dist available")
    #    print(item.children[0])
    elif count != 0:
        #print("row count", count, len(item.contents))
        #for subItem in item:
        #    print(len(subItem), subItem.string)
        #print("gender", item.contents[0].find('a').attrs['data-query-gender'])
        #print("race distance and stroke", item.contents[0])
        #print("race distance and stroke", item.contents[0].find('span').find('a').contents[0])
        #print("race round", item.contents[1].contents[0])
        #print("race time", item.contents[3].contents[0])
        #get the gender for both swim and race dict
        sw_gender = item.contents[0].find('a').attrs['data-query-gender']
        temp_rc_dict['sw_id'] = sw_id
        temp_rc_dict['sw_yob'] = sw_yob
        temp_rc_dict['sw_gender'] = sw_gender
        #temp_sw_dict['sw_gender'] = sw_gender
        temp_rc_dict['rc_dist_stroke'] = item.contents[0].find('span').find('a').contents[0]
        temp_rc_dict['rc_round'] = item.contents[1].contents[0]
        temp_rc_dict['rc_time'] = item.contents[3].contents[0]
        race_list.append(temp_rc_dict)
    else:
        print("row count", count)
    
    #append the temp dict to the lists
    #swimmer_list.append(temp_sw_dict)
    #race_list.append(temp_rc_dict)
    count += 1
    # just print out some to see the pattern
    #if count == 11:
    #    break

In [ ]:
swimmer_list

In [ ]:
race_list